
<!--
%\VignetteEngine{knitr::rmarkdown}
%\VignetteIndexEntry{Using PaxtoolsR}
%\VignetteKeywords{paxtoolsr}
%\VignetteDepends{paxtoolsr}
%\VignettePackage{paxtoolsr}
-->

# PaxtoolsR Tutorial



In [ ]:
library(knitr)
opts_chunk$set(out.extra='style="display:block; margin: auto"', fig.align="center", tidy=TRUE)



In [ ]:
BiocStyle::markdown()



The **paxtoolsr** package exposes a number of the algorithms and functions provided by the Paxtools Java library and Pathway Commons webservice allowing them to be used in R.

# Overview

## BioPAX, Paxtools, Pathway Commons, and the Simple Interaction Format

The [Biological Pathway Exchange](http://www.ncbi.nlm.nih.gov/pubmed/20829833) (BioPAX) format is a community-driven standard language to represent biological pathways at the molecular and cellular level and to facilitate the exchange of pathway data. BioPAX can represent metabolic and signaling pathways, molecular and genetic interactions and gene regulation networks. Using BioPAX, millions of interactions, organized into thousands of pathways, from many organisms are available from a growing number of databases. This large amount of pathway data in a computable form will support visualization, analysis and biological discovery. The BioPAX format using syntax for data exchange based on the OWL ([Web Ontology Language](http://www.w3.org/TR/owl2-overview/)) that aids pathway data integration; classes in the BioPAX ontology are described [here](http://www.biopax.org/owldoc/Level3/). Ontologies are formal systems for knowledge representation allowing machine-readability of pathway data; one well-known example of a biological ontology is the [Gene Ontology](http://geneontology.org) for biological terms.

[Paxtools](http://www.ncbi.nlm.nih.gov/pubmed/24068901) is a Java libary that allows users to interact with biological pathways represented in the BioPAX language. [Pathway Commons](http://www.pathwaycommons.org/about/) is a resource that integrates biological pathway information for a number of public pathway databases, including Reactome, PantherDB, HumanCyc, etc. that are represented using the BioPAX language.

**NOTE:** BioPAX can encode very detailed information about biological processes. Analysis of this data, however, can be complicated as one needs to consider a wide array of n-ary relationships, different states of entities and generics. An alternative approach is to derive higher order relations based on a set of templates to define a simple binary network between biological entities and use conventional graph algorithms to analyze it. For many users of this package, the binary representation termed the Simple Interaction Format (SIF) will be the main entry point to the usage of BioPAX data. Conversion of BioPAX datasets to the SIF format is done through a series of [simplification rules](https://docs.google.com/document/d/1coFo66uuPQQ4ZMSHr8IzCV7I2DwXCoDBfZw7Vg4MgUE/edit?usp=sharing).

## Limitations

The Paxtools Java library produces that full model of a given BioPAX data set that can be searched via code. The **paxtoolsr** provides a limited set of functionality mainly to produce SIF representations of networks that can be analyzed in R.

# Basics

## Installation



In [ ]:
source("http://bioconductor.org/biocLite.R")
biocLite("paxtoolsr")



## Getting Started

Load **paxtoolsr** package:



In [ ]:
library(paxtoolsr)



A list of all accessible vignettes and methods is available with the following command:



In [ ]:
help.search("paxtoolsr")



For help on any **paxtoolsr** package functions, use one of the following command formats:



In [ ]:
help(graphPc)
?graphPc



## Common Function Return Types

**paxtoolsr** return two main types of values **data.frame** and **XMLInternalDocument**. Data.frames are table like data structures. **XMLInternalDocument** is a representation provided by the **XML** package and this data structure form is returned for functions that search or return raw BioPAX results. An **XMLInternalDocument** can be used as the input for any function requiring a BioPAX file.

# Handling BioPAX OWL Files

**paxtoolsr** provides several functions for handling BioPAX OWL files. paxtoolsr provides several functions for handling BioPAX OWL files: merging, validation, conversion to other formats. Many databases with protein-protein interactions and pathway information export the BioPAX format and BioPAX files; databases that support the BioPAX format can be found on [PathGuide](http://pathguide.org/), a resource for pathway information.

## Merging BioPAX Files

We illustrate how to merge two BioPAX files. Only entities that share IDs will be merged; no additional merging occurs on cross-references. The merging occurs as described further in the Java library [documentation](http://sourceforge.net/projects/biopax/files/paxtools/paxtools.pdf/download). Throughout this tutorial we use the system.file() command to access sample BioPAX files included with the **paxtoolsr** package. Merging may result in warning messages caused as a result of redundant actions being checked against by the Java library; these messages may be ignored.



In [ ]:
file1 <- system.file("extdata", "raf_map_kinase_cascade_reactome.owl", package="paxtoolsr")
file2 <- system.file("extdata", "biopax3-short-metabolic-pathway.owl", package="paxtoolsr")

mergedFile <- mergeBiopax(file1, file2)



Here we summarize information about one of the BioPAX files provide in the **paxtoolsr** package. The summarize() function produces a counts for various BioPAX classes and can be used to filter through BioPAX files matching particular characteristics. In the example below, we show that the merged file contains the sum of the Catalysis elements from the original two BioPAX files. This can be used iterate over and to identify files with particular properties quickly or to summarize across the files from a set.



In [ ]:
s1 <- summarize(file1)
s2 <- summarize(file2)
s3 <- summarize(mergedFile)

s1$Catalysis
s2$Catalysis
s3$Catalysis



## Validating BioPAX Files

To validate BioPAX **paxtoolsr** the types of validation performed are described in the [**BioPAX Validator**](http://www.ncbi.nlm.nih.gov/pubmed/23918249) publication by Rodchenkov I, et al.



In [ ]:
errorLog <- validate(system.file("extdata", "raf_map_kinase_cascade_reactome.owl", package="paxtoolsr"), onlyErrors=TRUE)



## Converting BioPAX Files to Other Formats

It is often useful to convert BioPAX into other formats. Currently, **paxtoolsr** supports conversion to [Gene Set Enrichment Analysis](http://www.broadinstitute.org/gsea/index.jsp) (GSEA, .gmt), [Systems Biology Graphical Notation](http://www.sbgn.org/) (SBGN, .sbgn), [Simple Interaction Format](SIF).

### Simple Interaction Format (SIF) Network

The basic SIF format includes a three columns: PARTICIPANT_A, INTERACTION_TYPE, and PARTICIPANT_B; possible INTERACTION_TYPEs are described [here](http://www.pathwaycommons.org/pc/sif_interaction_rules.do).



In [ ]:
sif <- toSif(system.file("extdata", "biopax3-short-metabolic-pathway.owl", package="paxtoolsr"))



SIF representations of networks are returned as **data.frame** objects. SIF representations can be readily be visualized in network analysis tools, such as [Cytoscape](http://www.cytoscape.org), which can be interfaced with through the R package, [RCytoscape](http://www.bioconductor.org/packages/release/bioc/html/RCytoscape.html).



In [ ]:
head(sif)



### Extended Simple Interaction Format (SIF) Network

Often analysis requires additional items of information, this could be the literature references related to a resource or the name of the data source where an interaction was derived. This information can be retrieved as part of an extended SIF network. A BioPAX dataset can be converted to extended SIF network.



In [ ]:
# Select additional node and edge properties
inputFile <- system.file("extdata", "raf_map_kinase_cascade_reactome.owl", package="paxtoolsr")

results <- toSifnx(inputFile=inputFile)



The **results** object is a list with two entries: nodes and edges. **nodes** will be a **data.table** where each row corresponds to a biological entity, an **EntityReference**, and will contain any user-selected node properties as additional columns. Similarly, **edges** will be a **data.table** with a SIF extended with any user-selected properties for an **Interaction** as additional columns. Information on possible properties for an EntityReference or Interaction is available through the [BioPAX ontology](http://www.biopax.org/owldoc/Level3/). It is also possible to download a pre-computed extended SIF representation for the entire [Pathway Commons database](http://www.pathwaycommons.org) that includes information about the data sources for interactions and identifiers for nodes; refer to documentation of the method for more details about the returned entries.

**NOTE:** Conversion of **results** entries from **data.table** to **data.frame** can be done using **setDF** in the **data.table** package.

**NOTE:** **downloadPc** may take several minutes to complete. It is suggested that the results of this command be saved locally rather than using this command frequently.



In [ ]:
results <- downloadPc(format="SIFNX")



# Searching Pathway Commons

Networks can also be loaded using Pathway Commons rather than from local BioPAX files. First, we show how Pathway Commons can be searched.



In [ ]:
## Search Pathway Commons for "glycolysis"-related pathways
searchResults <- searchPc(q="glycolysis", type="pathway")



All functions that query Pathway Commons include a flag **verbose** that allows users to see the query URL sent to Pathway Commons for debugging purposes.



In [ ]:
## Search Pathway Commons for "glycolysis"-related pathways
searchResults <- searchPc(q="glycolysis", type="pathway", verbose=TRUE)



Pathway Commons search results are returned as an XML object.



In [ ]:
str(searchResults)



These results can be filtered using the **XML** package using XPath expressions; [examples of XPath expressions and syntax](http://www.w3schools.com/xpath/xpath_examples.asp). The examples here shows how to pull the name for the pathway and the URI that contains information about the pathway in the BioPAX format.



In [ ]:
xpathSApply(searchResults, "/searchResponse/searchHit/name", xmlValue)[1]
xpathSApply(searchResults, "/searchResponse/searchHit/pathway", xmlValue)[1]



Alternatively, these XML results can be converted to data.frames using the **XML** and **plyr** libraries.



In [ ]:
library(plyr)
searchResultsDf <- ldply(xmlToList(searchResults), data.frame)

# Simplified results
simplifiedSearchResultsDf <- searchResultsDf[, c("name", "uri", "biopaxClass")]



This type of searching can be used to locally save BioPAX files retrieved from Pathway Commons.



In [ ]:
## Use an XPath expression to extract the results of interest. In this case, the URIs (IDs) for the pathways from the results
searchResults <- xpathSApply(searchResults, "/searchResponse/searchHit/uri", xmlValue)

## Generate temporary file to save content into
biopaxFile <- tempfile()

## Extract a URI for a pathway in the search results and save into a file
idx <- which(grepl("panther", simplifiedSearchResultsDf$uri) &
             grepl("glycolysis", simplifiedSearchResultsDf$name, ignore.case=TRUE))
uri <- simplifiedSearchResultsDf$uri[idx]
saveXML(getPc(uri, "BIOPAX"), biopaxFile)



# Extracting Information from BioPAX Datasets Using traverse()

The **traverse** function allows the extraction of specific entries from BioPAX records. traverse() functionality should be available for any **uniprot.org** or **purl.org** URI.



In [ ]:
# Get the protein Uniprot ID for a given HGNC gene symbol
id <- idMapping("AKT1")

# Covert the Uniprot ID to a URI that would be found in Pathway Commons
uri <- paste0("http://identifiers.org/uniprot/", id)

# Get URIs for only the ModificationFeatures of the protein
xml <- traverse(uri=uri, path="ProteinReference/entityFeature:ModificationFeature")

# Extract all the URIs
uris <- xpathSApply(xml, "//value/text()", xmlValue)

# For the first URI get the modification position and type
tmpXml <- traverse(uri=uris[1], path="ModificationFeature/featureLocation:SequenceSite/sequencePosition")
cat("Modification Position: ", xpathSApply(tmpXml, "//value/text()", xmlValue))

tmpXml <- traverse(uri=uris[1], path="ModificationFeature/modificationType/term")
cat("Modification Type: ", xpathSApply(tmpXml, "//value/text()", xmlValue))



# Common Data Visualization Pathways and Network Analysis

## Visualizing SIF Interactions from Pathway Commons using R Graph Libraries

A common use case for **paxtoolsr** to retrieve a network or sub-network from a pathway derived from a BioPAX file or a Pathway Commons query. Next, we show how to visualize subnetworks loaded from BioPAX files and retrieved using the Pathway Commons webservice. To do this, we use the **igraph** R graph library because it has simple methods for loading edgelists, analyzing the networks, and visualizing these networks.

Next, we show how subnetworks queried from Pathway Commons can be visualized directly in R using the **igraph** library. Alternatively, these graphical plots can be made using [Cytoscape](http://cytoscape.org) either by exporting the SIF format and then importing the SIF format into Cytoscape or by using the **RCytoscape** package to work with Cytoscape directly from R.



In [ ]:
library(igraph)



We load the network from a BioPAX file using the SIF format:



In [ ]:
sif <- toSif(system.file("extdata", "biopax3-short-metabolic-pathway.owl", package="paxtoolsr"))

# graph.edgelist requires a matrix
g <- graph.edgelist(as.matrix(sif[,c(1,3)]), directed=FALSE)
plot(g, layout=layout.fruchterman.reingold)



## Pathway Commons Graph Query

Next, we show how to perform graph search using Pathway Commons useful for finding connections and neighborhoods of elements. This can be used to extract the neighborhood of a single gene that is then filtered for a specific interaction type: "controls-state-change-of". State change here indicates a modification of another molecule (e.g. post-translational modifications). This interaction type is directed, and it is read as "A controls a state change of B".



In [ ]:
gene <- "SIRT1"
t1 <- results <- graphPc(source=gene, kind="neighborhood", format="BINARY_SIF")
t2 <- t1[which(t1[,2] == "controls-state-change-of"),]

g <- graph.edgelist(as.matrix(t2[,c(1,3)]), directed=FALSE)
plot(g, layout=layout.fruchterman.reingold)



The example below shows the extraction of a sub-network connecting a set of proteins:



In [ ]:
genes <- c("AKT1", "IRS1", "MTOR", "IGF1R")
t1 <- graphPc(source=genes,
              kind="PATHSBETWEEN",
              format="BINARY_SIF",
              verbose=TRUE)
t2 <- t1[which(t1[,2] == "controls-state-change-of"),]

g <- graph.edgelist(as.matrix(t2[,c(1,3)]), directed=FALSE)
plot(g, layout=layout.fruchterman.reingold)



## Overlaying Experimental Data on Pathway Commons Networks

Often, it is useful not only to visualize a biological pathway, but also to overlay a given network with some form of biological data, such as gene expression values for genes in the network.



In [ ]:
library(RColorBrewer)

# Generate a color palette that goes from white to red that contains 10 colors
numColors <- 10
colors <- colorRampPalette(brewer.pal(9, "Reds"))(numColors)

# Generate values that could represent some experimental values
values <- runif(length(V(g)$name))

# Scale values to generate indicies from the color palette
xrange <- range(values)
newrange <- c(1, numColors)

factor <- (newrange[2]-newrange[1]) / (xrange[2]-xrange[1])
scaledValues <- newrange[1] + (values-xrange[1]) * factor
indicies <- as.integer(scaledValues)

# Color the nodes based using the indicies and the color palette created above
g <- set.vertex.attribute(g, "color", value=colors[indicies])

#get.vertex.attribute(h, "color")

plot(g, layout = layout.fruchterman.reingold)



## Network Statistics

Often it is useful to produce statistics on a network. Here we show how to determine SIF network statistics and statistics on BioPAX files.

### SIF Network Statistics

Once Pathway Commons and BioPAX networks are loaded as graphs using the **igraph** R package, it is possible to analyze these networks. Here we show how to get common statistics for the current network retrieved from Pathway Commons:



In [ ]:
# Degree for each node in the igraph network
degree(g)
#Number of nodes
length(V(g)$name)
#Clustering coefficient
transitivity(g)
#Network density
graph.density(g)
# Network diameter
diameter(g)



Another common task determine paths between nodes in a network.



In [ ]:
# Get the first shortest path between two nodes
tmp <- get.shortest.paths(g, from="IRS1", to="MTOR")

# igraph seems to return different objects on Linux versus OS X for get.shortest.paths()
if(class(tmp[[1]]) == "list") {
	path <- tmp[[1]][[1]]	# Linux
} else {
  path <- tmp[[1]] # OS X
}

# Convert from indicies to vertex names
V(g)$name[path]



# Gene Set Enrichment Analysis with Pathway Commons

The processing of the microarray data is taken from the following webpage: [Bioconductor Tutorial on Microarray Processing and Gene Set Analysis](http://www.bioconductor.org/help/course-materials/2005/BioC2005/labs/lab01/estrogen/) with for grabbing gene sets from a Pathway Commons pathway and using same data as in the example, but stored in the **estrogen** R package.

To access microarray data sets, users should consider retrieving data from the NCBI Gene Expression Omnibus (GEO) using the [GEOQuery package](http://www.bioconductor.org/packages/2.13/bioc/html/GEOquery.html).

The first thing we'll do is load up the necessary packages.



In [ ]:
library(paxtoolsr) # To retrieve data from Pathway Commons
library(limma) # Contains geneSetTest
library(affy) # To load microarray data
library(hgu95av2) # Annotation packages for the hgu95av2 platform
library(hgu95av2cdf)
library(XML) # To parse XML files



We then retrieve a pathway of interest using the the Pathway Commons search functionality.



In [ ]:
# Generate a Gene Set
## Search Pathway Commons for "glycolysis"-related pathways
searchResults <- searchPc(q="glycolysis", type="pathway")

## Use an XPath expression to extract the results of interest. In this case, the URIs (IDs) for the pathways from the results
searchResults <- xpathSApply(searchResults, "/searchResponse/searchHit/uri", xmlValue)

## Generate temporary files to save content into
biopaxFile <- tempfile()

## Extract the URI for the first pathway in the search results and save into a file
uri <- searchResults[2]
saveXML(getPc(uri, "BIOPAX"), biopaxFile)



And then, we convert this pathway to a gene set.



In [ ]:
## Generate temporary files to save content into
gseaFile <- tempfile()

## Generate a gene set for the BioPAX pathway with gene symbols
### NOTE: Not all search results are guaranteed to result in gene sets
tmp <- toGSEA(biopaxFile, gseaFile, "HGNC Symbol", FALSE)
geneSet <- tmp$geneSet



Finally, we process the microarray data and apply the gene set entrichment analysis.



In [ ]:
# Process Microarray Data
## Load/process the estrogen microarray data
estrogenDataDir <- system.file("extdata", package="estrogen")
targets <- readTargets(file.path(estrogenDataDir, "estrogen.txt"), sep="")

abatch <- ReadAffy(filenames=file.path(estrogenDataDir, targets$filename))
eset <- rma(abatch)

f <- paste(targets$estrogen,targets$time.h,sep="")
f <- factor(f)
design <- model.matrix(~0+f)
colnames(design) <- levels(f)

fit <- lmFit(eset, design)

cont.matrix <- makeContrasts(E10="present10-absent10",E48="present48-absent48",Time="absent48-absent10",levels=design)

fit2  <- contrasts.fit(fit, cont.matrix)
fit2  <- eBayes(fit2)

## Map the gene symbols to the probe IDs
symbol <- unlist(as.list(hgu95av2SYMBOL))

### Check that the gene symbols are on the microarray platform
genesOnChip <- match(geneSet,symbol)
genesOnChip # CHECK FOR ERROR HERE

genesOnChip <- genesOnChip[!is.na(genesOnChip)]

### Grab the probe IDs for the genes present
genesOnChip <- names(symbol[genesOnChip])
genesOnChip <- match(genesOnChip, rownames(fit2$t))
genesOnChip <- genesOnChip[!is.na(genesOnChip)]

## Run the Gene Set Test from the limma Package
geneSetTest(genesOnChip,fit2$t[,1],"two.sided")



# ID Mapping

Functions and results from **paxtoolsr** functions can be used in conjunction with the ID mapping functions of the [**biomaRt** Bioconductor package](http://www.bioconductor.org/packages/release/bioc/html/biomaRt.html); users should check the the **biomaRt** package documentation for a list of possible ID types.



In [ ]:
sif <- toSif(system.file("extdata", "raf_map_kinase_cascade_reactome.owl", package="paxtoolsr"))

# Generate a mapping between the HGNC symbols in the SIF to the Uniprot IDs
library(biomaRt)
ensembl <- useMart("ensembl")
ensembl <- useDataset("hsapiens_gene_ensembl", mart=ensembl)

hgnc_symbol <- c(sif$PARTICIPANT_A, sif$PARTICIPANT_B)
output <- getBM(attributes=c('hgnc_symbol', 'uniprot_sptrembl'), filters='hgnc_symbol', values=hgnc_symbol, mart=ensembl)

# Remove blank entries
output <- output[output[,2] != "",]



# Troubleshooting

## File Paths

Use properly delimited and full paths (do not use relative paths, such as ../directory/file or ~/directory/file) to files should be used with the **paxtoolsr** package.



In [ ]:
toSif("/directory/file")
#or
toSif("X:\\directory\\file")



## Memory Limits: Specify JVM Maximum Heap Size

By default **paxtoolsr** uses a maximum heap size limit of 512MB. For large BioPAX files, this limit may be insufficient. The code below shows how to change this limit and observe that the change was made.

**NOTE:** This limit cannot be changed once the virtual machine has been initialized
by loading the library, so the memory heap size limit must be changed beforehand.



In [ ]:
options(java.parameters="-Xmx1024m")

library(paxtoolsr)

# Megabyte size
mbSize <- 1048576.0

runtime <- .jcall("java/lang/Runtime", "Ljava/lang/Runtime;", "getRuntime")
maxMemory <- .jcall(runtime, "J", "maxMemory")
maxMemoryMb <- maxMemory / mbSize
cat("Max Memory: ", maxMemoryMb, "\n")



# Session Information



In [ ]:
sessionInfo()



# References

* Cerami EG, Gross BE, Demir E, Rodchenkov I, Babur O, Anwar N, Schultz N, Bader GD, Sander C. Pathway Commons, a web resource for biological pathway data. Nucleic Acids Res. 2011 Jan;39(Database issue):D685-90. doi: 10.1093/nar/gkq1039. Epub 2010 Nov 10.
* Rodchenkov I, Demir E, Sander C, Bader GD. The BioPAX Validator. Bioinformatics. 2013 Oct 15;29(20):2659-60. doi: 10.1093/bioinformatics/btt452. Epub 2013 Aug 5.
